In [4]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


import os 
os.chdir("..")

In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from numerics.integration.steps import *
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from numerics.utilities.misc import *

In [6]:
pars = give_def_params_discrimination(flip=0) ## H0 is the true hypothesis
params, exp_path = check_params(pars)

In [43]:
times, logliks, states1, states0, signals, covs1, covs0 =load_data_discrimination(itraj=20,
                                                                                  total_time = 4, 
                                                                                  dt=1e-6, exp_path = exp_path)

In [13]:
st = np.load("analysis/plot_data_stopping_time/stopp_time_1K.npy")

In [19]:
def stop_time(itraj, a=-3, b=3):
    pars = give_def_params_discrimination(flip=0) ## H0 is the true hypothesis
    params, exp_path = check_params(pars)
    times, logliks, states1, states0, signals, covs1, covs0 =load_data_discrimination(itraj=itraj, total_time = 4, dt=1e-6, exp_path = exp_path)
    l0,l1 = logliks[:,0], logliks[:,1]
    log_lik_ratio = l1 - l0  ###
    stopping_time_ind =np.argmin(np.logical_and(log_lik_ratio < b, log_lik_ratio > a))
    return times[stopping_time_ind]

In [18]:
ind_time_sprt_avg_stop_time = np.argmin(np.abs(times - np.mean(st)))

In [21]:
ind_time_sprt_avg_stop_time

845066

In [66]:
stprt_stop = {}
stprt_stop["h0"] = []
stprt_stop["h1"] = []

np_errs = {}
np_errs["h0/h1"] = []
np_errs["h1/h0"] = []

sprt_errs = {}
sprt_errs["h0/h1"] = []
sprt_errs["h1/h0"] = []

In [67]:
for itraj in tqdm(range(1000)):
    try:
        pars = give_def_params_discrimination(flip=0) ## H0 is the true hypothesis
        params, exp_path = check_params(pars)
        logliks =load_data_discrimination_liks(itraj=itraj, total_time = 4, dt=1e-6, exp_path = exp_path)
        l0,l1 = logliks[:,0], logliks[:,1]
        log_lik_ratio = l1 - l0  ###
        
        ### stand up at the equal likelihood point ---> you choose max-likelihood hypothesis!
        h0 = log_lik_ratio[ind_time_sprt_avg_stop_time] <= 0
        h0_sprt = log_lik_ratio[ind_time_sprt_avg_stop_time] <= -3
        
        sprt_errs["h0/h1"].append(int(h0_sprt))
        np_errs["h0/h1"].append(int(h0))

        #### for the other hypothesis, flip the parameters
        pars = give_def_params_discrimination(flip=1)
        params, exp_path = check_params(pars)
        logliks =load_data_discrimination_liks(itraj=itraj, total_time = 4, dt=1e-6, exp_path = exp_path)
        ### logliks ---> p(yt | \thetaNull)  , p(yt | thetaTrue )
        ### now we want to see the p(H1 | H0) ---> check how many times we'd guess for h1 if data generated by h0 at stopping time
        log_lik_ratio_swap = logliks[:,0] - logliks[:,1]  ### since i swapped the parameters, this gives p(y_t | h1)/p(y_t | h0)   (now in the case where i generate y_t under h0)

        ### stand up at the equal likelihood point ---> you choose max-likelihood hypothesis!
        h1 = log_lik_ratio_swap[ind_time_sprt_avg_stop_time] >= 0
        h1_sprt = log_lik_ratio_swap[ind_time_sprt_avg_stop_time] >= 3
        
        np_errs["h1/h0"].append(int(h1))
        sprt_errs["h1/h0"].append(int(h1_sprt))
        
    except Exception:
        pass


100%|██████████| 1000/1000 [08:59<00:00,  1.85it/s]


In [70]:
errs_np = [np.mean(np_errs["h0/h1"]), np.mean(np_errs["h1/h0"])]
print(errs_np)

errs_sprt = [np.mean(sprt_errs["h0/h1"]), np.mean(sprt_errs["h1/h0"])]
print(errs_sprt)
#print(np.mean(sprt_errs["h0/h1"]), np.mean(sprt_errs["h1/h0"]))

[0.022110552763819097, 0.017346938775510204]
[0.0020100502512562816, 0.003061224489795918]


In [74]:
perr_np = 0.5*np.sum(errs_np)
perr_np

0.01972874576966465

In [75]:
perr_sprt = 0.5*np.sum(errs_sprt)
perr_sprt

0.0025356373705260997

In [76]:
perr_np/perr_sprt

7.780586450960567

In [86]:
perr_sprt**(1/4)

0.22439944887472782

In [87]:
perr_np

0.01972874576966465

In [82]:
t=1
b=3
[gamma1, omega1, n1, eta1, kappa1], [gamma0, omega0, n0, eta0, kappa0] = params
_, mu = ggiulio_prob(t, b, kappa0, kappa1, eta0 , eta1, n0, n1, gamma0, gamma1)

10.798530454048887

In [78]:
def ggiulio_prob(t, b, kappa0, kappa1, eta0 , eta1, n0, n1, gamma0, gamma1):
    Su1 = n1 + 0.5 + (kappa1 / gamma1)
    Su0 = n0 + 0.5 + (kappa0 / gamma0)

    S1 = (np.sqrt(1 + (16.0*eta1*kappa1*Su1/gamma1)) - 1)*(gamma1/(8.0*eta1*kappa1))
    S0 = (np.sqrt(1 + (16.0*eta0*kappa0*Su0/gamma0)) - 1)*( gamma0/(8.0*eta0*kappa0))

    lam = gamma0 + (8*eta0*kappa0*S0)

    aa = (4*eta1*kappa1*(S1**2))/gamma1
    bb =(4*eta0*kappa0*S0**2)*(1+((16.0*eta1*kappa1*S1)/ (gamma1 + lam)) + (64.0*(eta1 * kappa1 * S1)**(2)/(gamma1 * (gamma1 + lam))))/ lam
    c =8 *(S0*S1*(eta0*kappa0 *eta1*kappa1)**(0.5)) * (gamma1+ (4.0*eta1*kappa1*S1) ) / ((gamma1 + lam)*gamma1)

    mu = 4*(eta1*kappa1*aa + (eta0*kappa0*bb) - 2*np.sqrt(eta1*kappa1*eta0*kappa0)*c)
    S= np.sqrt(2*mu)

    div = (np.sqrt(2*np.pi)*S*(t**(3/2)))
    return  abs(b)*np.exp(-((abs(b)-mu*t)**2)/(2*t*(S**2)))/div, mu

In [57]:
2*np.exp(-3)

0.09957413673572789

In [70]:
pars = give_def_params_discrimination(h1true=1)
params, exp_path = check_params(pars)

times, l0, l1, states, states1, signals, covs, covs1 = load_data_discrimination(exp_path = exp_path, periods = 2, ppp=100000,itraj=1)
log_lik_ratio = l0-l1

bools = np.logical_and(log_lik_ratio <= b(ep1), a(ep0) <= log_lik_ratio)
indi = np.argmin(bools)
stprt_stop["h1"] = times[indi]

h0 = log_lik_ratio[indi] <= a(ep0)

In [72]:
int(h0)

1

In [40]:
alpha = 0
beta = 0

for itraj in tqdm(range(1,10)):

    pars = give_def_params_discrimination(h1true=1)
    params, exp_path = check_params(pars)
    
    times, l0, l1, states, states1, signals, covs, covs1 = load_data(exp_path = exp_path, periods = 50, ppp=1000, itraj=itraj)
    log_lik_ratio = l0-l1

    bools = np.logical_and(log_lik_ratio <= b(ep1), a(ep0) <= log_lik_ratio)
    indi = np.argmin(bools)

    h1 = log_lik_ratio[indi] >= b(ep1)
    h0 = log_lik_ratio[indi] <= a(ep0)
    
    alpha += h0

  0%|                                                                                                                                                         | 0/9 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../quantera/trajectories/discrimination/[[1.0, 0, 20, 1.0, 1.0], [0.3, 0, 2, 1.0, 1.0]]/1itraj/rossler_real_traj_method/50periods/1000ppp/1rppp/states.npy'